<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [2]:
#проверка и установка psycopg2 и lxml
try:
    import psycopg2
except ImportError:
    !pip install psycopg2
    import psycopg2
    
try:
    import lxml
except ImportError:
    !pip install lxml
    import lxml

import pandas as pd

import warnings
# фильтр для сокрытия предупреждений SQLAlchemy connectable и категории FutureWarning, чтобы не загромождали вывод программы
warnings.filterwarnings('ignore', message = '.*SQLAlchemy connectable')
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.core.series")
warnings.simplefilter(action='ignore', category=FutureWarning)


In [3]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 0

In [ ]:
# Загрузка параметров подключения из файла .env - если не заданы выше в ячейке [3] 

if not (DBNAME and USER and PASSWORD and HOST and PORT):
    !pip install python-dotenv
    from dotenv import load_dotenv
    import os
    load_dotenv()
    DBNAME = os.getenv('DBNAME')
    USER = os.getenv('USER')
    PASSWORD = os.getenv('PASSWORD')
    HOST = os.getenv('HOST')
    PORT = os.getenv('PORT')

In [85]:
# установление соединения с базой данных PostgreSQL с помощью библиотеки psycopg2
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [6]:
# текст запроса
query_3_1 = f'''SELECT COUNT(v.id) as "количество вакансий"
                FROM public.vacancies v
'''

In [7]:
# результат запроса
display(pd.read_sql_query(query_3_1, connection))
# 	количество вакансий
#	              49197

,количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [8]:
# текст запроса
query_3_2 = f'''SELECT COUNT(e.id) as "количество работодателей"
                FROM public.employers e
'''

In [ ]:
# результат запроса
display(pd.read_sql_query(query_3_2, connection))
#
# 	количество работодателей
#	                   23501

,количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [10]:
# текст запроса
query_3_3 = f'''SELECT COUNT(a.id) as "количество регионов"
                FROM public.areas a
'''

In [ ]:
# результат запроса
display(pd.read_sql_query(query_3_3, connection))
#
# количество регионов
#	             1362

,количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [12]:
# текст запроса
query_3_4 = f'''SELECT COUNT(i.id) "количество сфер деятельности"
                FROM public.industries i
'''

In [ ]:
# результат запроса
display(pd.read_sql_query(query_3_4, connection))
#
# количество сфер деятельности
#                          294

,количество сфер деятельности
0,294


***

# Выводы по предварительному анализу данных
В базе данных содержится 49197 вакансий, количество работодателей 23501, количество регионов 1362, количество сфер деятельности 294. Данные представлены в таблицах с указанием вакансий, работодателей, регионов и сфер деятельности. Таблицы взаимосвязаны между собой полям id, что позволяет провести анализ данных, в том числе выявить взаимосвязи их характеристики.

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [14]:
# текст запроса
query_4_1 = f'''SELECT 
                    a.name area,
                    COUNT(v.id) cnt
                FROM public.vacancies v
                JOIN public.areas a ON a.id = v.area_id
                GROUP BY a.name
                ORDER BY cnt DESC
'''

In [15]:
# результат запроса
display('Количество вакансий в каждом регионе (ТОП-5):')
display(pd.read_sql_query(query_4_1, connection).head(5))
#
#'Количество вакансий в каждом регионе (ТОП-5):'
#              area 	     cnt
# 0	         Москва	        5333
# 1	Санкт-Петербург	        2851
# 2	          Минск	        2112
# 3	    Новосибирск	        2006
# 4	         Алматы	        1892

'Количество вакансий в каждом регионе (ТОП-5):'

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [16]:
# текст запроса
query_4_2 = f'''SELECT 
                    COUNT(v.id) as "Количество вакансий с зарплатой"
                FROM public.vacancies v
                WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL                
                ORDER BY COUNT(v.id) DESC
'''

In [17]:
# результат запроса
display('Количества вакансий заполнено хотя бы одно из двух полей с зарплатой')
display(pd.read_sql_query(query_4_2, connection))
#
# 'Количества вакансий заполнено хотя бы одно из двух полей с зарплатой'
#Количество вакансий с зарплатой
#	                       24073
#

'Количества вакансий заполнено хотя бы одно из двух полей с зарплатой'

,Количество вакансий с зарплатой
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [18]:
# текст запроса
query_4_3 = f'''SELECT 
                    ROUND(AVG(v.salary_from)) AS "среднее нижней границы зарплаты",
                    ROUND(AVG(v.salary_to)) AS "среднее верхней границы зарплаты"
                FROM public.vacancies v
'''

In [19]:
# результат запроса
display(pd.read_sql_query(query_4_3, connection))
# 
# среднее нижней границы зарплаты	среднее верхней границы зарплаты
# 	                      71065.0	                        110537.0


,среднее нижней границы зарплаты,среднее верхней границы зарплаты
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [20]:
# текст запроса
query_4_4 = f'''SELECT 
                    v.schedule as "Тип рабочего графика",
                    v.employment as "Тип трудоустройства",
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                GROUP BY v.schedule, v.employment
                ORDER BY COUNT(v.id) DESC
'''

In [21]:
# результат запроса
display(pd.read_sql_query(query_4_4, connection))
#
#   Тип рабочего графика    Тип трудоустройства	    Количество вакансий
# 0	Полный день	            Полная занятость	        35367
# 1	Удаленная работа	    Полная занятость	        7802
# 2	Гибкий график	        Полная занятость	        1593
# 3	Удаленная работа	    Частичная занятость	        1312
# 4	Сменный график	        Полная занятость	        940
# 5	Полный день	            Стажировка	                569

,Тип рабочего графика,Тип трудоустройства,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [22]:
# текст запроса
query_4_5 = f'''SELECT 
                    v.experience as "Требуемый опыт работы",
                    COUNT(v.experience) as "Количество вакансий"
                FROM public.vacancies v
                GROUP BY v.experience
                ORDER BY COUNT(v.experience) ASC
'''

In [23]:
# результат запроса
display(pd.read_sql_query(query_4_5, connection))
#
# 	Требуемый опыт работы	Количество вакансий
# 0	Более 6 лет	                1337
# 1	Нет опыта	                7197
# 2	От 3 до 6 лет	            14511
# 3	От 1 года до 3 лет	        26152

,Требуемый опыт работы,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

# Выводы по детальному анализу вакансий

Анализ данных выявил, что спрос на трудовые ресурсы сосредоточен в крупных городах. Количество вакансий с указанной зарплатой составляет 49%, при этом средние уровни для зарплатных вилок указанных в вакансиях составляют от 70 до 110 тысяч рублей.
Большинство вакансий включает полную занятость с полным днём работы (72%), на втором месте - полную занятость с удаленным графиком работы (16%). Наиболее востребованные вакансии - с опытом работы от 1 года до 3 лет (53%) и от 3 до 6 лет (29%). Также отмечается небольшое количество вакансий без требований к опыту работы (15%), а также крайне незначительное количество вакансий с высокими требованиями к опыту более 6 лет (3%).

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
# текст запроса
query_5_1 = f'''(SELECT 
                    e.name as "Работодатель",
                    COUNT(v.id) as "Количество вакансий"
                FROM public.employers e
                JOIN public.vacancies v ON e.id = v.employer_id
                GROUP BY e.name
                ORDER BY COUNT(v.id) DESC
                LIMIT 1)
                UNION ALL
                (SELECT 
                    e.name as "Работодатель",
                    COUNT(v.id) as "Количество вакансий"
                FROM public.employers e
                JOIN public.vacancies v ON e.id = v.employer_id
                GROUP BY e.name
                ORDER BY COUNT(v.id) DESC
                OFFSET 4 LIMIT 1)                
'''

In [25]:
# результат запроса
display(f'Работодатели, находящиеся на первом и пятом местах по количеству вакансий:')
display(pd.read_sql_query(query_5_1, connection))
#
# 'Работодатели, находящиеся на первом и пятом местах по количеству вакансий:'
#       Работодатель	Количество вакансий
# 0	          Яндекс	               1933
# 1	   Газпром нефть	                331

'Работодатели, находящиеся на первом и пятом местах по количеству вакансий:'

,Работодатель,Количество вакансий
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
# текст запроса
query_5_2 = f'''SELECT
                    a.name AS "Название региона",
                    COUNT(DISTINCT e.id) AS "Количество работодателей",
                    COUNT(DISTINCT v.id) AS "Количество вакансий"
                FROM public.areas a
                JOIN public.employers e ON e.area = a.id 
                LEFT JOIN public.VACANCIES v ON v.area_id = a.id
                WHERE v.id is NULL             
                GROUP BY a.name
                ORDER BY 2 DESC
                LIMIT 1
'''

In [27]:
# результат запроса
display(pd.read_sql_query(query_5_2, connection))
#
#	Название региона	Количество работодателей	Количество вакансий
# 0	Россия	            410	                        0

,Название региона,Количество работодателей,Количество вакансий
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
query_5_3 = f'''SELECT
                    e.name AS "Название работодателя",
                    COUNT(DISTINCT v.area_id) AS "Количество регионов с вакансиями"
                FROM public.employers e 
                LEFT JOIN public.vacancies v ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY 2 DESC
'''

In [29]:
# результат запроса
display(pd.read_sql_query(query_5_3, connection))
#
# 	Название работодателя	Количество регионов с вакансиями
# 0	Яндекс	                    181
# 1	Ростелеком	                152
# 2	Спецремонт	                116
# 3	Поляков Денис Иванович	    88
# 4	ООО ЕФИН	                71
# ...


,Название работодателя,Количество регионов с вакансиями
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [31]:
# текст запроса
query_5_4 = f'''SELECT
                    COUNT(DISTINCT e.id) AS empl_wo_industry
                FROM public.employers e 
                LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
                WHERE ei.industry_id IS NULL     
'''

In [32]:
# результат запроса
display('Количество работодателей, у которых не указана сфера деятельности')
display(pd.read_sql_query(query_5_4, connection))
#
# 'Количество работодателей, у которых не указана сфера деятельности'
# empl_wo_industry
# 0	        8419

'Количество работодателей, у которых не указана сфера деятельности'

,empl_wo_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [33]:
# текст запроса
query_5_5 = f'''SELECT
                    e.name AS "Название работодателя",
                    COUNT(DISTINCT ei.industry_id) AS "Количество сфер деятельности"
                FROM public.employers e 
                LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
                GROUP BY e.name
                HAVING COUNT(DISTINCT ei.industry_id) = 4
                ORDER BY 1 ASC
                OFFSET 2 LIMIT 1 
'''

In [34]:
# результат запроса
display('Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности:')
display(pd.read_sql_query(query_5_5, connection))
#
#
# 'Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности:'
# Название работодателя	Количество сфер деятельности
# 0	            2ГИС	                           4

'Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности:'

,Название работодателя,Количество сфер деятельности
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [35]:
# текст запроса
query_5_6 = f'''SELECT
                    COUNT(DISTINCT e.id) AS "Количество работодателей"
                FROM public.employers e 
                LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
                JOIN public.industries i ON ei.industry_id = i.id
                WHERE i.name LIKE 'Разработка программного обеспечения'
'''

In [36]:
# результат запроса
display('Количество работодателей в качестве сферы деятельности указана Разработка программного обеспечения:')
display(pd.read_sql_query(query_5_6, connection))
#
# 'Количество работодателей в качестве сферы деятельности указана Разработка программного обеспечения:'
# Количество работодателей
# 0	        3553

'Количество работодателей в качестве сферы деятельности указана Разработка программного обеспечения:'

,Количество работодателей
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников
from urllib.parse import quote_plus
#
# получение из веб-источника кортежа с городами-миллионниками
# кодируем URL, заменяя специальные символы на их эквиваленты в формате URL
url = quote_plus('https://ru.wikipedia.org/wiki/Города-миллионеры_России', safe=':/')
# загружаем все таблицы с указанного URL и возвращаем их в виде списка DataFrame
tables = pd.read_html(url)

# кортеж с городами-миллионниками
cities_w_mln = tuple(tables[0]['Город'])
display(cities_w_mln)

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [38]:
# текст запроса
query_5_7 = f'''(SELECT
                    a.name AS "Название региона",
                    COUNT(DISTINCT v.id) AS "Количество вакансий"
                FROM 
                    public.vacancies v
                    JOIN public.areas a ON a.id = v.area_id
                    JOIN public.employers e ON e.id = v.employer_id
                GROUP BY 
                    v.area_id, a.name, e.id
                HAVING
                    a.name IN {(cities_w_mln)} AND e.name = 'Яндекс'
                ORDER BY 2)
                                
                UNION ALL
                
                (SELECT
                    'Total',
                    SUM(total.cnt)
                FROM
                    (
                    SELECT
                        count(*) AS cnt
                    FROM
                        public.vacancies AS v
                        JOIN public.employers AS e ON v.employer_id = e.id
                        JOIN public.areas AS a ON v.area_id = a.id
                    GROUP BY
                        v.area_id, a.name, e.id
                    HAVING
                        a.name IN {(cities_w_mln)} AND e.name = 'Яндекс'
                    ) AS total
                )
'''

In [39]:
# результат запроса
display('Список регионов-миллионников, в которых представлены вакансии компании Яндекс:')
display(pd.read_sql_query(query_5_7, connection))
#
# 'Список регионов-миллионников, в которых представлены вакансии компании Яндекс:'
# Название региона	Количество вакансий
# 0	    Омск	        21.0
# 1	    Челябинск	    22.0
# 2	    Красноярск	    23.0
# 3	    Волгоград	    24.0
# 4	    Ростов-на-Дону	25.0
# 5	    Казань	        25.0
# 6	    Пермь	        25.0
# 7	    Самара	        26.0
# 8	    Уфа	            26.0
# 9	    Краснодар	    30.0
# 10	Воронеж	        32.0
# 11	Новосибирск	    35.0
# 12	Нижний Новгород	36.0
# 13	Екатеринбург	39.0
# 14	Санкт-Петербург	42.0
# 15	Москва	        54.0
# 16	Total	        485.0

'Список регионов-миллионников, в которых представлены вакансии компании Яндекс:'

,Название региона,Количество вакансий
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Ростов-на-Дону,25.0
5,Казань,25.0
6,Пермь,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

# Выводы по анализу работодателей
Анализ данных по работодателям показал, что больше всего вакансий размещают компании из ИТ, телекоммуникационного, финансового и нефтегазового секторов. Эти вакансии представлены в наибольшем количестве регионов. У 3553 (14%) работодателей в качестве сферы деятельности указана "Разработка программного обеспечения". При этом у 36% работодателей не указана сфера деятельности. Наибольшая концентрация вакансий наблюдается в городах с населением более 1 миллиона человек (48,5%).

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [40]:
# текст запроса
query_6_1 = f'''SELECT 
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                WHERE v.name ILIKE '%data%' OR v.name ILIKE '%данн%'
'''

In [41]:
# результат запроса
display('Количество вакансий, которые имеют отношение к данным:')
display(pd.read_sql_query(query_6_1, connection))
#
# 'Количество вакансий, которые имеют отношение к данным:'
# Количество вакансий
# 0     	     1771


'Количество вакансий, которые имеют отношение к данным:'

,Количество вакансий
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [42]:
# текст запроса
query_6_2 = f'''SELECT 
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
                    AND
                    (v.name ILIKE '%junior%' OR
                    v.experience = 'Нет опыта' OR
                    v.employment = 'Стажировка')                    
'''

In [43]:
# результат запроса
display('Количество вакансий, подходящих для начинающего дата-сайентиста:')
display(pd.read_sql_query(query_6_2, connection))
#
# 'Количество вакансий, подходящих для начинающего дата-сайентиста:'
# Количество вакансий
# 0	               51

'Количество вакансий, подходящих для начинающего дата-сайентиста:'

,Количество вакансий
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
# текст запроса
query_6_3 = f'''SELECT 
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
                    AND
                    (v.key_skills ILIKE '%SQL%' OR
                    v.key_skills ILIKE '%postgres%')                    
'''

In [45]:
# результат запроса
display('Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан SQL или postgres:')
display(pd.read_sql_query(query_6_3, connection))
#
# 'Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан SQL или postgres:'
# Количество вакансий
# 0	              229

'Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан SQL или postgres:'

,Количество вакансий
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [46]:
# текст запроса
query_6_4 = f'''SELECT 
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
                    AND
                    (v.key_skills ILIKE '%Python%')                    
'''

In [47]:
# результат запроса
display('Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python:')
display(pd.read_sql_query(query_6_4, connection))
#
# 'Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python:'
# Количество вакансий
# 0	              357

'Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python:'

,Количество вакансий
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [48]:
# текст запроса
query_6_5 = f'''SELECT 
                    ROUND(AVG(
                        LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, CHR(9), '')) + 1
                        ), 2) AS "Среднее количество навыков"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
'''

In [49]:
# результат запроса
display('Среднее значение количества ключевых навыков в вакансиях для DS:')
display(pd.read_sql_query(query_6_5, connection))
#
# 'Среднее значение количества ключевых навыков в вакансиях для DS:'
# Среднее количество навыков
# 0	                    6.41

'Среднее значение количества ключевых навыков в вакансиях для DS:'

,Среднее количество навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [50]:
# текст запроса
query_6_6 = f'''SELECT 
                    v.experience AS "Тип требуемого опыта",
                    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) AS "Средняя зарплата для DS"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') AND
                    (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                GROUP BY v.experience
'''

In [51]:
# результат запроса
display('Средняя зарплата для DS для каждого типа требуемого опыта:')
display(pd.read_sql_query(query_6_6, connection))
#
# 	Тип требуемого опыта	Средняя зарплата для DS
# 0	    Нет опыта	            74643.0
# 1	    От 1 года до 3 лет	    139675.0
# 2	    От 3 до 6 лет	        243115.0

'Средняя зарплата для DS для каждого типа требуемого опыта:'

,Тип требуемого опыта,Средняя зарплата для DS
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

# Выводы по предметному анализу

Общее количество вакансий, имеющих отношение к обработке данных составляет 1771 (3,6% от общего количества вакансий). Из них 51 вакансия подходит для начинающего дата-сайнтиста (2,88% от количества вакансий DS), что означает повышенные требования работодателей к наличию опыта. 

Наибольшее количество вакансий в DS соответствует опыту "От 1 года до 3 лет" - 223 (46,46%) и опыту "От 3 до 6 лет" - 195 (40,63%).

Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан SQL или postgres составляет 229 (12.93% от количества вакансий DS). Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python - 357 (20.16%  от количества вакансий DS). 

Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python, SQL или postgres - 408 (23,04% от количества вакансий DS). 

Среднее значение количества ключевых навыков в вакансиях для DS составило 6.41,  что отражает высокие требования работодателей к соискателям по квалификации и наличию опыта в нескольких областях. 

Средняя зарплата для специалиста DS в разрезе каждого типа требуемого опыта составила от  74643 рублей (Нет опыта), 139675 рублей (От 1 года до 3 лет) и 243115 рублей (От 3 до 6 лет опыта), что отражает существенную разницу в зарплатах в зависимости от опыта. 

Следует особо отметить, что количество вакансий в области DS c типом требуемого опыта "От 6 лет" составило 34, однако по всем этим вакансиям отсутствует значения заработной платой.

Примечание. Часть дополнительных расчетов приведена в разделе Дополнительные исследования.

***

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования


В рассматриваемых базах данных содержится значительное количество вакансий (49197), работодателей (23501), регионов (1362) и сфер деятельности (294). Это позволяет провести детальный анализ и выявить важные взаимосвязи.

Спрос на трудовые ресурсы сосредоточен в крупных городах, что подтверждается высоким количеством вакансий в этих регионах.
Половина вакансий (49%) содержит информацию о зарплате, со средними значениями от 70 до 110 тысяч рублей.

Большинство вакансий предполагает полную занятость с полным рабочим днем (72%) или удаленным графиком (16%).
Наиболее востребованы специалисты с опытом работы от 1 до 3 лет (53%) и от 3 до 6 лет (29%).

Ведущие отрасли, размещающие вакансии, включают ИТ, телекоммуникации, финансы и нефтегазовый сектор.
36% работодателей не указывают сферу деятельности, что может затруднять анализ.

Наибольшая концентрация вакансий наблюдается в городах с населением более 1 миллиона человек - 23856 вакансий (48,5% от общего количества вакансий).

Вакансии в сфере обработки данных составляют 1771, или 3,6% от общего количества вакансий.
Требования к опыту высоки: только 2,88% вакансий подходят для начинающих специалистов.
Наиболее востребованные навыки включают Python (20,16%) и SQL/Postgres (12,93%).

Среднее количество ключевых навыков, требуемых для вакансий в Data Science (DS), составляет 6,41, что указывает на высокие требования к квалификации.

Наибольшее количество вакансий в DS соответствует опыту "От 1 года до 3 лет" - 223 (46,46%) и опыту "От 3 до 6 лет" - 195 (40,63%).

Средняя зарплата для вакансий в DS варьируется в зависимости от опыта: от 74643 рублей для начинающих до 243115 рублей для специалистов с опытом от 3 до 6 лет. 
Следует особо отметить, что количество вакансий в области DS c типом требуемого опыта "От 6 лет" составило 34, однако по всем этим вакансиям отсутствует значения заработной платой (расчет в разделе Дополнительные исследования).

Наибольшее количество вакансий для специалостов в области DS предусматривает полную занятость (97,08%), в том числе - полный рабочий день (66,46%),удаленную работу (22,92%) и гибкий график (7,71%).

Количество вакансий в области Data Science составило 480, или 0,98% от общего числа вакансий. При этом наибольшая концентрация вакансий в Data Science наблюдается в городах с населением более 1 миллиона человек 379 вакансий (79,0% от количества вакансий в области DS), а наибольшая часть из них приходится на такие города -  Москва, Санкт-Петербург, Новосибирск, Нижний Новгород и Казань (соответствующие дополнительные расчеты приведены ниже).

Эти выводы помогут лучше понять текущие тенденции на рынке труда и требования работодателей, а также определить ключевые области для дальнейшего анализа и улучшения стратегии найма. 

Перспективным направлением продолжения исследования является анализ вышеуказанных параметров во времени, что позволит сделать выводы о тенденциях развития на рынке труда.

***

# Дополнительные исследования

### Альтернативное решение задания 5.1

In [ ]:
# текст запроса 
query_5_1_a = f'''SELECT 
                    e.name as "Работодатель",
                    COUNT(v.id) as "Количество вакансий"
                FROM public.employers e
                JOIN public.vacancies v ON e.id = v.employer_id
                GROUP BY e.name
                ORDER BY COUNT(v.id) DESC
                LIMIT 5
'''

In [56]:
# результат запроса

df_5_1_a = pd.read_sql_query(query_5_1_a, connection)
display(f'Работодатель, находящийся на первом месте по количеству вакансий: {df_5_1_a.iloc[0][0]}, вакансий {df_5_1_a.iloc[0][1]}')
display(f'Работодатель, находящийся на пятом  месте по количеству вакансий: {df_5_1_a.iloc[4][0]}, вакансий {df_5_1_a.iloc[4][1]}')

# 'Работодатель, находящийся на первом месте по количеству вакансий: Яндекс, вакансий 1933'
# 'Работодатель, находящийся на пятом  месте по количеству вакансий: Газпром нефть, вакансий 331'

display(df_5_1_a.head(5))

'Работодатель, находящийся на первом месте по количеству вакансий: Яндекс, вакансий 1933'

'Работодатель, находящийся на пятом  месте по количеству вакансий: Газпром нефть, вакансий 331'

,Работодатель,Количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


### Дополнительное изучение для формирования выводов - полный список регионов-миллионников и количество представленных в них вакансий.
Модифицирован запрос из задания 5.7

In [60]:
#текст запроса 
query_5_7a = f'''(SELECT
                    a.name AS "Название региона",
                    COUNT(DISTINCT v.id) AS "Количество вакансий"
                FROM 
                    public.vacancies v
                    JOIN public.areas a ON a.id = v.area_id
                    JOIN public.employers e ON e.id = v.employer_id
                GROUP BY 
                    v.area_id, a.name
                HAVING
                    a.name IN {(cities_w_mln)}
                ORDER BY 2)
                                
                UNION ALL
                
                (SELECT
                    'Total',
                    SUM(total.cnt)
                FROM
                    (
                    SELECT
                        count(*) AS cnt
                    FROM
                        public.vacancies AS v
                        JOIN public.employers AS e ON v.employer_id = e.id
                        JOIN public.areas AS a ON v.area_id = a.id
                    GROUP BY
                        v.area_id, a.name
                    HAVING
                        a.name IN {(cities_w_mln)}
                    ) AS total
                )
'''

In [61]:
# результат запроса
display('Список регионов-миллионников, в которых представлены вакансии')
display(pd.read_sql_query(query_5_7a, connection))

'Список регионов-миллионников, в которых представлены вакансии'

,Название региона,Количество вакансий
0,Волгоград,456.0
1,Омск,617.0
2,Уфа,767.0
3,Пермь,771.0
4,Челябинск,786.0
5,Красноярск,847.0
6,Воронеж,1063.0
7,Ростов-на-Дону,1131.0
8,Самара,1144.0
9,Краснодар,1301.0


### Проверка, насколько популярны Python, SQL и postgres в требованиях работодателей к DS. 
Вычисляем количество вакансий, в которых в качестве ключевого навыка указан Python, SQL или postgres.


In [62]:
# текст запроса
query_6_4_a = f'''SELECT 
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
                    AND
                    (v.key_skills ILIKE '%Python%' OR v.key_skills ILIKE '%SQL%' OR
                    v.key_skills ILIKE '%postgres%')                    
'''

In [63]:
# результат запроса
display('Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python, SQL или postgres:')
display(pd.read_sql_query(query_6_4_a, connection))
#
# 'Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python, SQL или postgres:'
# Количество вакансий
# 0	              408

'Количество вакансий, подходящих для DS, в которых в качестве ключевого навыка указан Python, SQL или postgres:'

,Количество вакансий
0,408


### Рассмотрим альтернативный расчет среднего значения количества ключевых навыков в вакансиях для DS, с учетом возможного наличия пропусков в данных. Исходим из предположения, что незаполненное поле "Ключевые навыки" соответствует значению 0 для расчета.

Дополнительный вариант, с учетом возможности v.key_skills IS NULL или v.key_skills = ''

In [66]:
# текст запроса
query_6_5_a = f'''SELECT 
                    ROUND(AVG(
                        CASE 
                            WHEN v.key_skills IS NULL OR v.key_skills = '' THEN 0
                            ELSE LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, CHR(9), '')) + 1
                        END 
                    ), 2) AS "Среднее количество навыков"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
'''

In [67]:
# результат дополнительного варианта решения задания 6.5
display('Среднее значение количества ключевых навыков в вакансиях для DS:')
display("С учетом возможности key_skills IS NULL или key_skills = ''")
display(pd.read_sql_query(query_6_5_a, connection))
#
# 'Среднее значение количества ключевых навыков в вакансиях для DS:'
#"С учетом возможности key_skills IS NULL или key_skills = ''"
# Среднее количество навыков
# 0	                    5.75


'Среднее значение количества ключевых навыков в вакансиях для DS:'

"С учетом возможности key_skills IS NULL или key_skills = ''"

,Среднее количество навыков
0,5.75


### Проверим - какие типы трудоустройства и рабочего графика соответствуют вакансиям для специалиста DS

In [68]:
# текст запроса
query_6_5_2 = f'''SELECT 
                    v.schedule as "Тип рабочего графика",
                    v.employment as "Тип трудоустройства",
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
                GROUP BY v.schedule, v.employment
                ORDER BY COUNT(v.id) DESC              
'''

In [69]:
# результат запроса
display('Структура вакансий для специалиста DS по типам трудоустройства и рабочего графика:')
display(pd.read_sql_query(query_6_5_2, connection))
#
# 'Структура вакансий для специалиста DS по типам трудоустройства и рабочего графика:'
#       Тип рабочего графика	        Тип трудоустройства	        Количество вакансий
# 0	        Полный день	                Полная занятость	        319
# 1	        Удаленная работа	        Полная занятость	        110
# 2	        Гибкий график	            Полная занятость	        37
# 3	        Гибкий график	            Частичная занятость	        4
# 4	        Удаленная работа	        Частичная занятость	        3
# 5	        Полный день	                Стажировка	                2
# 6	        Удаленная работа	        Проектная работа	        2
# 7	        Гибкий график	            Стажировка	                2
# 8	        Полный день	                Частичная занятость	        1

'Структура вакансий для специалиста DS по типам трудоустройства и рабочего графика:'

,Тип рабочего графика,Тип трудоустройства,Количество вакансий
0,Полный день,Полная занятость,319
1,Удаленная работа,Полная занятость,110
2,Гибкий график,Полная занятость,37
3,Гибкий график,Частичная занятость,4
4,Удаленная работа,Частичная занятость,3
5,Полный день,Стажировка,2
6,Удаленная работа,Проектная работа,2
7,Гибкий график,Стажировка,2
8,Полный день,Частичная занятость,1


### Дополнительное изучение - список регионов-миллионников и количество представленных в них вакансий для специалиста в области DS

In [70]:
# текст запроса
query_5_7_2 = f'''(SELECT
                    a.name AS "Название региона",
                    COUNT(DISTINCT v.id) AS "Количество вакансий"
                FROM 
                    public.vacancies v
                    JOIN public.areas a ON a.id = v.area_id
                    JOIN public.employers e ON e.id = v.employer_id
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
                GROUP BY 
                    a.name
                HAVING
                    a.name IN {(cities_w_mln)} 
                ORDER BY 2)
                                
                UNION ALL
                
                (SELECT
                    'Total',
                    SUM(total.cnt)
                FROM
                    (
                    SELECT
                        count(*) AS cnt
                    FROM
                        public.vacancies AS v
                        JOIN public.employers AS e ON v.employer_id = e.id
                        JOIN public.areas AS a ON v.area_id = a.id
                    WHERE 
                        (v.name ILIKE '%data scientist%' OR 
                        v.name ILIKE '%data science%' OR
                        v.name ILIKE '%исследователь данных%' OR
                        (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                        v.name ILIKE '%machine learning%' OR
                        v.name ILIKE '%машинн%обучен%%') 
                    GROUP BY
                        a.name
                    HAVING
                        a.name IN {(cities_w_mln)}
                    ) AS total
                )
'''

In [71]:
# результат запроса
display('Структура вакансий для специалиста DS в разрезе регионов-миллионников:')
display(pd.read_sql_query(query_5_7_2, connection))

'Структура вакансий для специалиста DS в разрезе регионов-миллионников:'

,Название региона,Количество вакансий
0,Волгоград,1.0
1,Красноярск,1.0
2,Челябинск,1.0
3,Воронеж,2.0
4,Уфа,2.0
5,Омск,3.0
6,Пермь,5.0
7,Самара,5.0
8,Ростов-на-Дону,6.0
9,Краснодар,6.0


### Получим общее количество вакансий для специалиста в области DS.

In [92]:
# текст запроса
query_6_2_2 = f'''SELECT 
                    COUNT(v.id) as "Количество вакансий"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%')               
'''
# примечание
# в задании проекта № 6.2 по рекомендации менторов использованы ILIKE '%ML%'
# в других расчетах использовано более корректное выражение (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR 
# В случае использоввании варианта (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
# будет получен результат:
# 'Количество вакансий для специалиста DS:'
#   Количество вакансий
# 0	                536

In [93]:
# результат запроса
display('Количество вакансий для специалиста DS:')
display(pd.read_sql_query(query_6_2_2, connection))
#
# 'Количество вакансий для специалиста DS:'
#   Количество вакансий
# 0	                480

'Количество вакансий для специалиста DS:'

,Количество вакансий
0,480


### Вычисляем, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта. 

Дополнительное поле - количество вакансий. Расчет включает и значения NULL в зарплатной вилке

In [94]:
# текст запроса
query_6_6_2 = f'''SELECT 
                    v.experience AS "Тип требуемого опыта",
                    COUNT(v.id) AS "Количество вакансий",
                    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) AS "Средняя зарплата для DS"
                FROM public.vacancies v
                WHERE 
                    (v.name ILIKE '%data scientist%' OR 
                    v.name ILIKE '%data science%' OR
                    v.name ILIKE '%исследователь данных%' OR
                    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                    v.name ILIKE '%machine learning%' OR
                    v.name ILIKE '%машинн%обучен%%') 
                GROUP BY v.experience
'''

In [95]:
# результат запроса
display('Средняя зарплата для DS для каждого типа требуемого опыта:')
display(pd.read_sql_query(query_6_6_2, connection))
#

#       Тип требуемого опыта	Количество вакансий	Средняя зарплата для DS
# 0	        Более 6 лет	                34	            NaN
# 1	        Нет опыта	                28	            74643.0
# 2	        От 1 года до 3 лет	        223	            139675.0
# 3	        От 3 до 6 лет	            195	            243115.0


'Средняя зарплата для DS для каждого типа требуемого опыта:'

,Тип требуемого опыта,Количество вакансий,Средняя зарплата для DS
0,Более 6 лет,34,NaN
1,Нет опыта,28,74643.0
2,От 1 года до 3 лет,223,139675.0
3,От 3 до 6 лет,195,243115.0


In [96]:
# не забываем закрыть соединение после окончания работы
connection.close()